# 实验 4.2：成员推理攻击

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 解释成员推理攻击的基本原理
- ✅ 理解模型置信度与训练集成员身份的关系
- ✅ 实现基于置信度阈值的成员推理
- ✅ 分析过拟合对攻击成功率的影响

## 📚 前置知识

- 神经网络分类的基本概念
- 模型训练与过拟合
- Softmax 概率输出的含义

## 🖥️ 实验环境

- 平台：腾讯 Cloud Studio
- GPU：NVIDIA Tesla T4（16GB）
- 模型：自定义 PyTorch 分类网络

## 📝 填空说明

本实验共 **4 个填空**，难度：⭐⭐⭐☆☆

⚠️ **安全提醒**：本实验仅用于教育目的，演示如何防护模型隐私泄露。请勿将技术用于未授权系统。

## 第一部分：核心概念与环境准备

### 什么是成员推理攻击？

**核心思想**：判断一个样本是否被用于训练模型

**类比理解**：
- 你的模型在考试题上准确率高，在考试题外的题上准确率低
- 攻击者可以推断："这道题准确率这么高，你肯定在训练时见过它"

**隐私风险**：
- 攻击者可以推断数据是否在训练集中
- 对于敏感数据集（医疗、财务），这是严重的隐私泄露

### 关键观察

在过拟合的模型中：
- **训练样本（成员）**：模型高置信度、准确率高
- **测试样本（非成员）**：模型低置信度、准确率低

这个差异可被用于成员推理攻击！

In [ ]:
# 本实验所需依赖
%pip install torch numpy matplotlib -q

# 验证安装
import torch
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch: {torch.__version__}")
print(f"NumPy: {np.__version__}")
print("✓ 依赖安装成功！")

# 设置随机种子（保证结果可复现）
torch.manual_seed(42)
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")

In [ ]:
import torch.nn as nn
import torch.optim as optim

# 创建一个简单的分类数据集
def create_dataset(n_samples=1000, n_features=20, n_classes=5):
    """
    创建一个合成分类数据集
    
    参数:
        n_samples: 样本数量
        n_features: 特征维度
        n_classes: 类别数
    
    返回:
        X: 特征矩阵 (n_samples, n_features)
        y: 标签向量 (n_samples,)
    """
    X = np.random.randn(n_samples, n_features).astype(np.float32)
    
    # 为每个类别创建中心点（使数据可分离）
    centers = np.random.randn(n_classes, n_features) * 2
    y = np.random.randint(0, n_classes, n_samples)
    
    # 将样本向对应类别中心偏移
    for i in range(n_samples):
        X[i] += centers[y[i]]
    
    return torch.tensor(X), torch.tensor(y)

# 创建数据集
X, y = create_dataset(n_samples=1000, n_features=20, n_classes=5)

# 划分训练集和测试集
train_size = 500
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

print("数据集创建完成：")
print(f"  总样本数: {len(X)}")
print(f"  特征维度: {X.shape[1]}")
print(f"  类别数: {len(torch.unique(y))}")
print(f"\n  训练集（成员）: {len(X_train)} 样本")
print(f"  测试集（非成员）: {len(X_test)} 样本")

In [ ]:
# 定义神经网络分类器
class SimpleClassifier(nn.Module):
    """
    简单的前馈神经网络分类器
    """
    def __init__(self, n_features=20, n_classes=5, hidden_size=64):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_classes)
        )
    
    def forward(self, x):
        return self.network(x)

print("模型架构：")
model_demo = SimpleClassifier()
print(model_demo)
print("\n✓ 模型定义完成！")

## 第二部分：训练目标模型

💡 **实验设计**

我们故意让模型过拟合，这样可以清楚地演示攻击效果：
- 训练集准确率高（模型"记住"了数据）
- 测试集准确率低（泛化能力差）
- 这个差异可被用于成员推理

In [ ]:
def train_model(model, X_train, y_train, epochs=50, lr=0.01):
    """
    训练分类模型
    
    参数:
        model: PyTorch 模型
        X_train: 训练特征
        y_train: 训练标签
        epochs: 训练轮数
        lr: 学习率
    
    返回:
        损失历史
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        if (epoch + 1) % 10 == 0:
            with torch.no_grad():
                acc = (outputs.argmax(dim=1) == y_train).float().mean()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Acc: {acc:.2%}")
    
    return losses

# 训练一个过拟合的模型
print("训练目标模型...")
print("(故意过拟合以演示攻击效果)\n")
print("=" * 50)

model = SimpleClassifier()
losses = train_model(model, X_train, y_train, epochs=100, lr=0.01)

print("=" * 50)

In [ ]:
# 评估模型在训练集和测试集上的表现
model.eval()

with torch.no_grad():
    train_outputs = model(X_train)
    train_acc = (train_outputs.argmax(dim=1) == y_train).float().mean()
    
    test_outputs = model(X_test)
    test_acc = (test_outputs.argmax(dim=1) == y_test).float().mean()

overfit_gap = train_acc - test_acc

print("\n" + "=" * 50)
print("模型性能评估")
print("=" * 50)
print(f"训练集准确率: {train_acc:.2%}")
print(f"测试集准确率: {test_acc:.2%}")
print(f"过拟合程度: {overfit_gap:.2%}")

if overfit_gap > 0.1:
    print(f"\n⚠️ 模型存在明显过拟合！")
    print(f"这使得成员推理攻击更容易成功。")
else:
    print(f"\n✓ 模型泛化性能良好，攻击难度会增加")

## 第三部分：实现成员推理攻击

💡 **攻击原理**

1. 获取模型对每个样本的置信度（对真实类别的概率）
2. 选择一个阈值（通常为训练集和测试集置信度的中位数）
3. 置信度高 → 推断为成员；置信度低 → 推断为非成员
4. 计算攻击准确率

**关键洞察**：过拟合导致成员和非成员的置信度分布差异大，使攻击更容易成功

In [ ]:
# ========== 填空 1：实现置信度计算 ==========
#
# 🎯 任务：从模型输出中提取对真实类别的置信度
#
# 💡 提示：
#   - softmax 将输出转换为概率分布
#   - gather(1, indices) 按索引提取特定位置的值
#   - 需要用 y.unsqueeze(1) 将标签变成列向量
#
# 请将 ___________ 替换为正确的代码
#
# 难度：⭐⭐☆☆☆

def get_confidence(model, X, y):
    """
    获取模型对每个样本的预测置信度
    
    参数:
        model: 训练好的模型
        X: 输入特征
        y: 真实标签
    
    返回:
        confidences: 每个样本的置信度 (numpy array)
    """
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        
        # 计算 softmax 概率分布
        probs = torch.softmax(outputs, dim=1)
        
        # 提取每个样本在真实类别上的置信度
        confidences = ___________  # 期望：probs.gather(1, y.unsqueeze(1)).squeeze()
    
    return confidences.numpy()

# 获取训练集和测试集的置信度
train_confidences = get_confidence(model, X_train, y_train)
test_confidences = get_confidence(model, X_test, y_test)

print("置信度统计：")
print("=" * 50)
print(f"训练集（成员）：")
print(f"  平均置信度: {np.mean(train_confidences):.4f}")
print(f"  中位数: {np.median(train_confidences):.4f}")
print(f"  标准差: {np.std(train_confidences):.4f}")

print(f"\n测试集（非成员）：")
print(f"  平均置信度: {np.mean(test_confidences):.4f}")
print(f"  中位数: {np.median(test_confidences):.4f}")
print(f"  标准差: {np.std(test_confidences):.4f}")

print(f"\n置信度差异: {np.mean(train_confidences) - np.mean(test_confidences):.4f}")

In [ ]:
# 可视化置信度分布
plt.figure(figsize=(11, 5))

plt.hist(train_confidences, bins=30, alpha=0.6, label='训练集（成员）', color='#FF6B6B', edgecolor='black')
plt.hist(test_confidences, bins=30, alpha=0.6, label='测试集（非成员）', color='#4ECDC4', edgecolor='black')

plt.xlabel('置信度', fontsize=11, fontweight='bold')
plt.ylabel('样本数量', fontsize=11, fontweight='bold')
plt.title('成员推理：置信度分布对比\n（左侧=低置信度，右侧=高置信度）', fontsize=13, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 关键观察：")
print("  成员样本的置信度分布偏向右侧（高置信度）")
print("  非成员样本的置信度分布偏向左侧（低置信度）")
print("  这个分布差异是攻击的基础！")

In [ ]:
# ========== 填空 2：实现成员推理函数 ==========
#
# 🎯 任务：根据阈值判断样本是否为成员
#
# 💡 提示：
#   - 置信度 > 阈值 → 预测为成员（返回 True）
#   - 置信度 <= 阈值 → 预测为非成员（返回 False）
#
# 请将 ___________ 替换为正确的代码
#
# 难度：⭐☆☆☆☆

def membership_inference_attack(confidences, threshold):
    """
    执行成员推理攻击
    
    参数:
        confidences: 模型对样本的置信度
        threshold: 成员/非成员的分界阈值
    
    返回:
        predictions: 布尔数组，True=成员，False=非成员
    """
    # 根据阈值判断成员身份
    predictions = ___________  # 期望：confidences > threshold
    
    return predictions

# 选择阈值（使用两个分布中位数的均值）
threshold = (np.median(train_confidences) + np.median(test_confidences)) / 2

print(f"选择的攻击阈值: {threshold:.4f}")
print(f"  训练集中位数: {np.median(train_confidences):.4f}")
print(f"  测试集中位数: {np.median(test_confidences):.4f}")

In [ ]:
# ========== 填空 3：计算攻击准确率 ==========
#
# 🎯 任务：计算成员的识别准确率
#
# 💡 提示：
#   - 成员应该被预测为 True（高置信度）
#   - 使用 np.mean() 计算正确预测的比例
#   - train_pred == True 返回布尔数组
#
# 请将 ___________ 替换为正确的代码
#
# 难度：⭐⭐☆☆☆

# 对训练集（成员）和测试集（非成员）执行攻击
train_pred = membership_inference_attack(train_confidences, threshold)
test_pred = membership_inference_attack(test_confidences, threshold)

# 计算攻击准确率
# 对成员：预测为 True 的准确率
# 对非成员：预测为 False 的准确率
member_acc = ___________    # 期望：np.mean(train_pred == True)
non_member_acc = np.mean(test_pred == False)

overall_acc = (member_acc + non_member_acc) / 2

print("\n成员推理攻击结果：")
print("=" * 50)
print(f"成员识别率（真阳率）: {member_acc:.2%}")
print(f"非成员识别率（真阴率）: {non_member_acc:.2%}")
print(f"总体准确率: {overall_acc:.2%}")

if overall_acc > 0.6:
    print(f"\n⚠️ 攻击成功！准确率显著高于随机猜测(50%)")
    print(f"差值: +{overall_acc - 0.5:.1%}")
else:
    print(f"\n✓ 攻击效果有限")

## 第四部分：阈值对攻击效果的影响

### 问题：最优阈值是什么？

不同阈值会影响：
- **真阳率（TPR）**：成员被正确识别的比例
- **假阳率（FPR）**：非成员被错误识别为成员的比例
- **总体准确率**：平衡的分类性能

In [ ]:
# 测试不同阈值的攻击效果
thresholds = np.linspace(0.1, 0.95, 20)
results = []

for thresh in thresholds:
    train_pred = membership_inference_attack(train_confidences, thresh)
    test_pred = membership_inference_attack(test_confidences, thresh)
    
    tpr = np.mean(train_pred == True)      # 真阳率：成员识别率
    fpr = np.mean(test_pred == True)       # 假阳率：非成员误判率
    acc = (tpr + (1 - fpr)) / 2            # 平衡准确率
    
    results.append((thresh, tpr, fpr, acc))

# 提取数据用于可视化
thresholds_list = [r[0] for r in results]
tpr_list = [r[1] for r in results]
fpr_list = [r[2] for r in results]
acc_list = [r[3] for r in results]

# 找到最佳阈值
best_idx = np.argmax(acc_list)
best_threshold = thresholds_list[best_idx]
best_acc = acc_list[best_idx]

print(f"最佳阈值分析：")
print("=" * 50)
print(f"最佳阈值: {best_threshold:.3f}")
print(f"最高准确率: {best_acc:.2%}")
print(f"对应的真阳率: {tpr_list[best_idx]:.2%}")
print(f"对应的假阳率: {fpr_list[best_idx]:.2%}")

In [ ]:
# 可视化阈值影响
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 左图：TPR vs FPR
ax1.plot(thresholds_list, tpr_list, 'o-', color='#FF6B6B', linewidth=2, markersize=5, label='真阳率（成员识别）')
ax1.plot(thresholds_list, fpr_list, 's-', color='#4ECDC4', linewidth=2, markersize=5, label='假阳率（非成员误判）')
ax1.axvline(x=best_threshold, color='gray', linestyle='--', alpha=0.5, label='最佳阈值')
ax1.set_xlabel('阈值', fontsize=11, fontweight='bold')
ax1.set_ylabel('比率', fontsize=11, fontweight='bold')
ax1.set_title('阈值 vs 真阳率/假阳率', fontsize=12, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# 右图：准确率
ax2.plot(thresholds_list, acc_list, 'D-', color='#95E1D3', linewidth=2.5, markersize=6)
ax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='随机猜测(50%)')
ax2.axvline(x=best_threshold, color='#FF6B6B', linestyle='--', alpha=0.7, label=f'最佳阈值({best_threshold:.2f})')
ax2.scatter([best_threshold], [best_acc], color='#FF6B6B', s=200, zorder=5, marker='*')
ax2.set_xlabel('阈值', fontsize=11, fontweight='bold')
ax2.set_ylabel('准确率', fontsize=11, fontweight='bold')
ax2.set_title('阈值 vs 攻击准确率', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0.4, 1.0])

plt.tight_layout()
plt.show()

print(f"\n💡 观察：")
print(f"  阈值太低 → 大多数被预测为成员 → 高假阳率")
print(f"  阈值太高 → 大多数被预测为非成员 → 低真阳率")
print(f"  最佳阈值平衡两者，实现最高准确率")

## 第五部分：过拟合程度对攻击的影响

### 核心问题：过拟合为什么会导致隐私泄露？

**直觉理解**：
- 模型"记住"了训练数据 = 过拟合
- 对训练数据的置信度高
- 对测试数据的置信度低
- 这个差异能被用于推断成员身份

In [ ]:
# ========== 填空 4：分析过拟合对攻击的影响 ==========
#
# 🎯 任务：观察并分析"过拟合程度越高 = 攻击成功率越高"的规律
#
# 💡 提示：
#   - 训练不同轮数的模型会有不同的过拟合程度
#   - 对比过拟合程度（train_acc - test_acc）和攻击准确率
#   - 这是一个实验观察任务，运行代码并分析结果
#
# 难度：⭐⭐☆☆☆（观察分析为主）

# 测试不同训练程度对攻击的影响
epoch_values = [10, 30, 50, 100, 150]
attack_results = []

print("测试不同训练程度对攻击成功率的影响...\n")
print("=" * 60)
print(f"{'轮数':<6} {'训练准确率':<10} {'测试准确率':<10} {'过拟合':<8} {'攻击准确率':<10}")
print("=" * 60)

for epochs in epoch_values:
    # 训练新模型
    temp_model = SimpleClassifier()
    train_model(temp_model, X_train, y_train, epochs=epochs, lr=0.01)
    
    # 评估过拟合程度
    temp_model.eval()
    with torch.no_grad():
        train_acc = (temp_model(X_train).argmax(dim=1) == y_train).float().mean().item()
        test_acc = (temp_model(X_test).argmax(dim=1) == y_test).float().mean().item()
    
    # 执行攻击
    train_conf = get_confidence(temp_model, X_train, y_train)
    test_conf = get_confidence(temp_model, X_test, y_test)
    best_thresh = (np.median(train_conf) + np.median(test_conf)) / 2
    
    train_pred = membership_inference_attack(train_conf, best_thresh)
    test_pred = membership_inference_attack(test_conf, best_thresh)
    attack_acc = (np.mean(train_pred == True) + np.mean(test_pred == False)) / 2
    
    overfit = train_acc - test_acc
    attack_results.append({
        'epochs': epochs,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'overfit': overfit,
        'attack_acc': attack_acc
    })
    
    print(f"{epochs:<6} {train_acc:<10.1%} {test_acc:<10.1%} {overfit:<8.1%} {attack_acc:<10.1%}")

print("=" * 60)

In [ ]:
# 可视化过拟合与攻击的关系
overfits = [r['overfit'] for r in attack_results]
attack_accs = [r['attack_acc'] for r in attack_results]
epochs_list = [r['epochs'] for r in attack_results]

fig, ax = plt.subplots(figsize=(10, 6))

# 散点图
scatter = ax.scatter(overfits, attack_accs, s=200, c=epochs_list, cmap='viridis', 
                     edgecolors='black', alpha=0.8, linewidth=2)

# 标注
for i, r in enumerate(attack_results):
    ax.annotate(f"{r['epochs']}e", 
               (overfits[i], attack_accs[i]),
               textcoords="offset points",
               xytext=(8, 8),
               fontsize=10,
               fontweight='bold')

# 参考线
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, alpha=0.5, label='随机猜测(50%)')

# 标签和标题
ax.set_xlabel('过拟合程度（训练准确率 - 测试准确率）', fontsize=12, fontweight='bold')
ax.set_ylabel('成员推理攻击准确率', fontsize=12, fontweight='bold')
ax.set_title('过拟合程度 vs 成员推理攻击成功率\n（更多过拟合 = 攻击更容易成功）', 
            fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

# 颜色条
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('训练轮数', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 关键发现：")
print(f"  过拟合程度最小: {min(overfits):.1%} → 攻击准确率: {attack_accs[overfits.index(min(overfits))]:.1%}")
print(f"  过拟合程度最大: {max(overfits):.1%} → 攻击准确率: {attack_accs[overfits.index(max(overfits))]:.1%}")
print(f"  相关性: 正相关 ✓")
print(f"\n💡 结论：")
print(f"  过拟合是隐私泄露的根本原因！")
print(f"  防止过拟合可以有效降低成员推理攻击成功率")

### 🤔 思考与讨论

1. **观察**：成员和非成员的置信度分布有什么区别？为什么会有这种区别？

2. **分析**：过拟合如何影响攻击成功率？从模型对训练数据的"记忆"角度解释。

3. **应用**：如何防御成员推理攻击？基于实验观察，提出可能的防御思路。
   - 减少过拟合（更强的正则化）
   - 添加噪声（差分隐私）
   - 限制输出信息（隐藏置信度）

4. **扩展**：除了置信度，还有其他指标可以用于成员推理吗？

## 📋 实验小结

### 核心收获

1. **概念理解**：
   - 成员推理攻击可以判断样本是否在训练集中
   - 过拟合导致成员和非成员的置信度差异明显
   - 这个差异可被利用进行隐私攻击

2. **技能掌握**：
   - 能从模型输出中提取置信度
   - 能选择合适的阈值进行分类
   - 能评估攻击的成功率

3. **安全认识**：
   - 模型隐私泄露是真实的安全威胁
   - 过拟合既降低泛化性能，又增加隐私风险
   - 需要从模型设计和训练方法多方面防护

### 关键代码回顾

```python
# 提取置信度（隐私泄露指标）
confidences = probs.gather(1, y.unsqueeze(1)).squeeze()

# 根据阈值推理成员身份
predictions = confidences > threshold

# 计算攻击成功率
attack_acc = (member_acc + non_member_acc) / 2
```

### 防御措施总结

| 防御方法 | 原理 | 优点 | 缺点 |
|---------|------|------|------|
| 正则化 | 减少过拟合 | 简单易行 | 可能影响模型准确率 |
| 差分隐私 | 训练时添加噪声 | 提供隐私保证 | 性能下降，复杂度高 |
| 输出过滤 | 隐藏/修改置信度 | 不影响模型 | 容易被绕过 |
| 成员预测检测 | 识别攻击行为 | 实时防护 | 检测率不一定高 |

### 延伸阅读

- [论文] Shokri et al. "Membership Inference Attacks Against Machine Learning Models" (S&P 2017)
- [防御方法] 实验 4.3：差分隐私训练
- [进阶话题] 模型反演攻击、属性推理攻击

## 📖 参考答案

<details>
<summary>点击查看参考答案</summary>

**填空 1**：提取置信度
```python
confidences = probs.gather(1, y.unsqueeze(1)).squeeze()
```

**填空 2**：根据阈值推理
```python
predictions = confidences > threshold
```

**填空 3**：计算准确率
```python
member_acc = np.mean(train_pred == True)
```

**填空 4**：分析过拟合影响（观察任务，无需填空）
- 运行并观察过拟合程度与攻击准确率的关系
- 预期：正相关关系

</details>

⚠️ **重要提醒**

实验完成后，请记得：

1. **释放 GPU 资源**：如果你使用的是 Tencent CloudStudio，请点击右上角的"停止"按钮，停止运行。
2. **保存实验结果**：如需保存，请下载 notebook 文件

这样可以避免不必要的资源占用和费用产生。**否则会一直消耗你的免费资源额度。**